# Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №2 - Дерево решений


**Общая информация**

**Срок сдачи:** до 27 марта 2018, 06:00   
**Штраф за опоздание:** -2 балла после 06:00 27 марта, -4 балла после 06:00 3 апреля, -6 баллов после 06:00 10 апреля

При отправлении ДЗ указывайте фамилию в названии файла   


Присылать ДЗ необходимо в виде ссылки на свой github репозиторий в slack @alkhamush
Необходимо в slack создать таск в приватный чат:   
/todo Фамилия Имя *ссылка на гитхаб* @alkhamush   
Пример:   
/todo Ксения Стройкова https://github.com/stroykova/spheremailru/stroykova_hw2.ipynb @alkhamush   

Используйте данный Ipython Notebook при оформлении домашнего задания.

###### Задание 1 (2 баллов)
Разберитесь в коде MyDecisionTreeClassifier, который уже частично реализован. В комментариях, где написано "Что делает этот блок кода?", ответьте на этот вопрос. Допишите код там, где написано "Ваш код". Ваша реализация дерева должна работать по точности не хуже DecisionTreeClassifier из sklearn. Точность проверяется на wine и Speed Dating Data.

###### Задание 2 (2 балла)
Добиться скорости работы на fit сравнимой со sklearn wine и Speed Dating Data. 
Для этого используем numpy. 

###### Задание 3 (2 балла)
Продемонстрируйте умение работать с Pipeline на данных Speed Dating Data и DecisionTreeClassifier. Нужно в pipeline произвести все необходимые преобразования данных и в конце обучить модель. Задание реализуйте под пунктом Задание 3 (уже написано ниже)

###### Задание 4 (2 балла)
Добавьте функционал, который определяет значения feature importance. Выведите 10 главных фичей под пунктом Задание 4 (уже написано ниже) для MyDecisionTreeClassifier и DecisionTreeClassifier так, чтобы сразу были видны выводы и по MyDecisionTreeClassifier, и по DecisionTreeClassifier. Используем данные Speed Dating Data.

###### Задание 5 (2 балла)
С помощью GridSearchCV или RandomSearchCV подберите наиболее оптимальные параметры для случайного леса (Выберете 2-3 параметра). Используем данные Speed Dating Data. Задание реализуйте под пунктом Задание 5 (уже написано ниже)


**Штрафные баллы:**

1. Невыполнение PEP8 -1 балл
2. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw2.ipynb) -1 балл
3. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -1 балл
4. При оформлении ДЗ нужно пользоваться данным файлом в качестве шаблона. Не нужно удалять и видоизменять написанный код и текст. В противном случае -1 балл

In [11]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline
#%load_ext pycodestyle_magic

In [12]:
class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=None,
                 sufficient_share=1.0, criterion='gini', max_features=None):
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        self.feature_importances_ = None
        if criterion == 'gini':
            self.G_function = self.__gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
        else:
            print('invalid criterion name')
            raise

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features is None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print('invalid max_features name')
            raise

    def __gini(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        return l_s * (1 - np.sum((l_c / l_s) ** 2, axis=1)[:, np.newaxis]) + \
            r_s * (1 - np.sum((r_c / r_s) ** 2, axis=1)[:, np.newaxis])

    def __entropy(self, l_c, l_s, r_c, r_s):
        return -(l_c * l_s * np.log(l_s)).sum(axis=1)[:, np.newaxis] - \
                (r_c * r_s * np.log(r_s)).sum(axis=1)[:, np.newaxis]

    def __misclass(self, l_c, l_s, r_c, r_s):
        return l_s * (1 - np.max(l_c / l_s, axis=1)[:, np.newaxis]) + \
               r_s * (1 - np.max(r_c / r_s, axis=1)[:, np.newaxis])

    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[k < np.sqrt(n_feature)]

    def __get_feature_ids_log2(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[k < np.log(n_feature) / np.log(2)]

    def __get_feature_ids_N(self, n_feature):
        return range(n_feature)

    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):
        # Что делает этот блок кода? - получает x и y в отсортированном
        # по признакам виде и затем получает количество классов
        sorted_x, sorted_y = self.__sort_samples(x, y)
        class_number = np.unique(y).shape[0]

        # Что делает этот блок кода? - разбивает выборку на подмножества
        # с учетом min_samples_split
        splitted_sorted_y = \
            sorted_y[self.min_samples_split:-self.min_samples_split]
        r_border_ids = \
            np.where(splitted_sorted_y[:-1] != splitted_sorted_y[1:])[0] + \
            (self.min_samples_split + 1)

        if len(r_border_ids) == 0:
            return float('+inf'), None

        # Что делает этот блок кода?- получаем массив количества одинаковых
        # элементов у, затем делаем one_hot_code
        # Делаем матрицу class_increments,
        # которая содержит количество каждого класса
        eq_el_count = r_border_ids - \
            np.append([self.min_samples_split], r_border_ids[:-1])
        one_hot_code = np.zeros((r_border_ids.shape[0], class_number))
        one_hot_code[np.arange(r_border_ids.shape[0]),
                     sorted_y[r_border_ids - 1]] = 1
        class_increments = one_hot_code * eq_el_count.reshape(-1, 1)
        class_increments[0] = class_increments[0] + np.bincount(
                                            sorted_y[:self.min_samples_split],
                                            minlength=class_number)

        # Что делает этот блок кода? - считаем кум. сумму для каждого класса
        # слева и справа, а также число элементов одного класса
        l_class_count = np.cumsum(class_increments, axis=0)
        r_class_count = np.bincount(sorted_y) - l_class_count
        l_sizes = r_border_ids.reshape(l_class_count.shape[0], 1)
        r_sizes = sorted_y.shape[0] - l_sizes

        # Что делает этот блок кода? - считает наименьшую impurities
        gs = self.G_function(l_class_count, l_sizes, r_class_count, r_sizes)
        idx = np.argmin(gs)

        # Что делает этот блок кода? - возвращает параметры оптимального
        # разбиения, а также значение impurity
        left_el_id = l_sizes[idx][0]
        return gs[idx], (sorted_x[left_el_id-1] + sorted_x[left_el_id]) / 2.0

    def __fit_node(self, x, y, node_id, depth, pred_f=-1):
        # Ваш код
        # Необходимо использовать следующее:
        # self.LEAF_TYPE
        # self.NON_LEAF_TYPE

        # self.tree
        # self.max_depth
        # self.sufficient_share
        # self.min_samples_split

        # self.get_feature_ids
        # self.__find_threshold
        # self.__div_samples
        # self.__fit_node

        length = x.shape[1]
        cnt = np.bincount(y, minlength=self.num_class)
        prob = cnt / np.sum(cnt)

        if (self.max_depth is not None and
           (depth == self.max_depth or x.shape[0] < self.min_samples_split)):
            self.tree[node_id] = (self.LEAF_TYPE, np.argmax(cnt), prob)
            return

        gs = np.zeros(length)
        thresholds = np.zeros(length)
        y -= np.min(y)

        for index in self.get_feature_ids(length):
            gs[index], thresholds[index] = \
                self.__find_threshold(x[:, index], y)

        y += np.min(y)

        if np.isnan(thresholds[np.argmin(gs)]):
            self.tree[node_id] = (self.LEAF_TYPE, np.argmax(cnt), prob)
            return

        x_l, x_r, y_l, y_r = self.__div_samples(x, y, np.argmin(gs),
                                                thresholds[np.argmin(gs)])

        if y_l.shape[0] == 0 or y_r.shape[0] == 0:
            self.tree[node_id] = (self.LEAF_TYPE, np.argmax(cnt), prob)
            return

        self.tree[node_id] = (self.NON_LEAF_TYPE, np.argmin(gs),
                              thresholds[np.argmin(gs)])

        self.__fit_node(x_l, y_l, 2 * node_id + 1, depth + 1)
        self.__fit_node(x_r, y_r, 2 * node_id + 2, depth + 1)

    def fit(self, x, y):
        self.num_class = np.unique(y).size
        self.__fit_node(x, y, 0, 0)

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]

    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])

    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)


In [13]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

In [14]:
wine = load_wine()
X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target,
                                                    test_size=0.1,
                                                    stratify=wine.target)


## Проверка скорости работы на wine

In [15]:
%time clf.fit(X_train, y_train)

CPU times: user 0 ns, sys: 8 ms, total: 8 ms
Wall time: 22.3 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [16]:
%time my_clf.fit(X_train, y_train)

CPU times: user 24 ms, sys: 4 ms, total: 28 ms
Wall time: 56.5 ms


## Проверка качества работы на wine

In [17]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.8917748917748917

In [18]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.9440559440559441

## Подготовка данных Speed Dating Data 

In [55]:
df = pd.read_csv('speed-dating-experiment/Speed Dating Data.csv',
                 encoding='cp1251')
df = df.iloc[:, :97]
df = df.drop(['id', 'idg', 'condtn', 'round', 'position', 'positin1',
              'order', 'partner', 'age_o', 'race_o', 'pf_o_att',
              'pf_o_sin', 'pf_o_int',
              'pf_o_fun', 'pf_o_amb', 'pf_o_sha',
              'dec_o', 'attr_o', 'sinc_o', 'intel_o', 'fun_o',
              'amb_o', 'shar_o', 'like_o', 'prob_o', 'met_o', 'field',
              'undergra', 'from', 'zipcode', 'career',
              'sports', 'tvsports', 'exercise', 'dining', 'museums',
              'art', 'hiking', 'gaming',
              'clubbing', 'reading', 'tv', 'theater', 'movies',
              'concerts', 'music', 'shopping', 'yoga',
              'expnum', 'wave'], axis=1)
df = df.dropna(subset=['age'])
df.loc[:, 'field_cd'] = df.loc[:, 'field_cd'].fillna(0)
df.loc[:, 'mn_sat'] = df.loc[:, 'mn_sat'].str.replace(
    ',', '').astype(np.float)
df.loc[:, 'mn_sat'] = df.mn_sat.fillna(-999)
df.loc[:, 'tuition'] = df.loc[:, 'tuition'].str.replace(
    ',', '').astype(np.float)
df.loc[:, 'tuition'] = df.tuition.fillna(-999)
df = df.dropna(subset=['imprelig', 'imprace'])
df.loc[:, 'income'] = df.loc[:, 'income'].str.replace(
    ',', '').astype(np.float)
df.loc[:, 'income'] = df.loc[:, 'income'].fillna(-999)
df = df.dropna(subset=['date'])
df.loc[:, 'career_c'] = df.loc[:, 'career_c'].fillna(0)
df.loc[:, 'temp_totalsum'] = df.loc[:, ['attr1_1', 'sinc1_1',
                                        'intel1_1', 'fun1_1',
                                        'amb1_1', 'shar1_1']].sum(
    axis=1)
df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1',
           'fun1_1', 'amb1_1', 'shar1_1']] = \
    (df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1',
                'amb1_1', 'shar1_1']].T/df.loc[
        :, 'temp_totalsum'].T).T * 100
df.loc[:, 'temp_totalsum'] = df.loc[:, ['attr2_1', 'sinc2_1',
                                        'intel2_1', 'fun2_1',
                                        'amb2_1', 'shar2_1']].sum(
    axis=1)
df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1',
           'fun2_1', 'amb2_1', 'shar2_1']] = \
    (df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1',
                'amb2_1', 'shar2_1']].T/df.loc[
        :, 'temp_totalsum'].T).T * 100
df = df.drop(['temp_totalsum'], axis=1)
for i in [4, 5]:
    feat = ['attr{}_1'.format(i), 'sinc{}_1'.format(i),
            'intel{}_1'.format(i), 'fun{}_1'.format(i),
            'amb{}_1'.format(i), 'shar{}_1'.format(i)]

    if i != 4:
        feat.remove('shar{}_1'.format(i))

    df = df.drop(feat, axis=1)
df_male = df.query('gender == 1').drop_duplicates(subset=[
    'iid', 'pid']).drop(['gender'], axis=1).dropna()
df_female = df.query('gender == 0').drop_duplicates(
    subset=['iid']).drop(['gender', 'match',
                          'int_corr', 'samerace'], axis=1).dropna()

df_female.columns = df_female.columns + '_f'
df_female = df_female.drop(['pid_f'], axis=1)
df_pair = df_male.join(df_female.set_index('iid_f'),
                       on='pid',
                       how='inner')
df_pair = df_pair.drop(['iid', 'pid'], axis=1)

In [56]:
X = df_pair.iloc[:, 1:].values
y = df_pair.iloc[:, 0].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

## Проверка скорости работы на Speed Dating Data 

In [57]:
%time clf.fit(X_train, y_train)

CPU times: user 120 ms, sys: 0 ns, total: 120 ms
Wall time: 163 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [58]:
%time my_clf.fit(X_train, y_train)

CPU times: user 1.74 s, sys: 8 ms, total: 1.74 s
Wall time: 2.05 s


## Проверка качества работы на Speed Dating Data

In [59]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.5774971297359357

In [60]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.5403472931562819

## Задание 3

In [67]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([('dtc', DecisionTreeClassifier())])

pipeline.fit(X_train, y_train)
f1_score(y_pred=pipeline.predict(X_test), y_true=y_test, average='macro')

0.5937881536758749

## Задание 4

In [68]:
model = DecisionTreeClassifier()
model.fit(X_train, y_train)

features_names = df_pair.columns[1:]
df = pd.Series(index=features_names,
               data=model.feature_importances_).sort_values(ascending=False)
df.head(10)


int_corr    0.064569
fun1_1      0.031598
fun1_1_f    0.027698
fun2_1_f    0.027686
date        0.024589
sinc1_1     0.024111
fun2_1      0.023922
amb1_1_f    0.023413
race_f      0.022881
exphappy    0.022785
dtype: float64

## Задание 5

In [70]:
parameters = {"max_depth": [2, 3, 4, 5, 6, 7, 8, None],
              "min_samples_split": range(2, 15),
              "criterion": ["entropy", "gini"]}

grid_search = RandomizedSearchCV(RandomForestClassifier(n_estimators=10),
                                 parameters, n_iter=10)
grid_search.fit(X_train, y_train)
grid_search.best_params_


{'criterion': 'entropy', 'max_depth': 7, 'min_samples_split': 7}